In [1]:
import os
import csv
import pandas as pd
import matplotlib.pyplot as plt
import json
import time
import scipy.stats as stats

mort = "./2012-2016 CardioVasc Mortality Rates.csv"
edu = "./Education.csv"
econo = "./SAIPESNC.csv"

In [103]:
mortality_file = pd.read_csv(mort)
education_file = pd.read_csv(edu)
socio_econo_file = pd.read_csv(econo)

In [104]:
education_file = education_file.rename(columns={'FIPS Code':'County ID'})
education_file.head()

,County ID,State,Area name,"Less than a high school diploma, 2012-2016","High school diploma only, 2012-2016","Some college or associate's degree, 2012-2016","Bachelor's degree or higher, 2012-2016","Percent of adults with less than a high school diploma, 2012-2016","Percent of adults with a high school diploma only, 2012-2016","Percent of adults completing some college or associate's degree, 2012-2016","Percent of adults with a bachelor's degree or higher, 2012-2016"
0,0,US,United States,"27,818,380","58,820,411","62,242,569","64,767,787",13.0,27.5,29.1,30.3
1,1001,AL,Autauga County,"4,528","12,519","10,451","8,968",12.4,34.3,28.7,24.6
2,1003,AL,Baldwin County,"13,956","40,154","44,486","41,350",10.0,28.7,31.8,29.5
3,1005,AL,Barbour County,"4,824","6,422","4,775","2,366",26.2,34.9,26.0,12.9
4,1007,AL,Bibb County,"3,040","6,586","4,234","1,890",19.3,41.8,26.9,12.0


In [105]:
new_edu_columns = ['County ID', 'State', "County Name", '< High School', 'High School', 'Associate',
                  'Bachelors <', 'Perc < High School', 'Perc High School', 'Perc Associate',
                  'Perc Bachelors <']
education_file.columns = new_edu_columns

In [106]:
edu_cleaned_columns = ['County ID', "County Name", 'Perc < High School', 'Perc High School', 'Perc Associate', 'Perc Bachelors <']
education_clean_df = education_file[edu_cleaned_columns]
education_clean_df['MYS'] = ""
education_clean_df.head()

C:\Users\corey\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


,County ID,County Name,Perc < High School,Perc High School,Perc Associate,Perc Bachelors <,MYS
0,0,United States,13.0,27.5,29.1,30.3,
1,1001,Autauga County,12.4,34.3,28.7,24.6,
2,1003,Baldwin County,10.0,28.7,31.8,29.5,
3,1005,Barbour County,26.2,34.9,26.0,12.9,
4,1007,Bibb County,19.3,41.8,26.9,12.0,


In [107]:
mean_years_schooling = []
for index, row in education_clean_df.iterrows():
    mean_years_schooling.append(((row['Perc < High School']*10) + (row['Perc High School'] * 12) + (row['Perc Associate'] * 14)
                              + (row['Perc Bachelors <'] * 20))/100)
    
education_clean_df['MYS'] = mean_years_schooling
education_clean_df['MYS'].describe(percentiles = [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9])

C:\Users\corey\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


count    3143.000000
mean       13.989138
std         0.834010
min        12.010000
10%        13.038400
20%        13.280800
30%        13.502000
40%        13.706000
50%        13.882000
60%        14.070000
70%        14.290800
80%        14.603200
90%        15.104800
max        18.602000
Name: MYS, dtype: float64

In [108]:
education_clean_df['Education Bin'] = ""

education_bins = [12, 13.05, 13.28, 13.502, 13.706, 13.882, 14.07, 14.291, 14.6032, 15.1048, 18.603]
education_bin_labels = ['1', '2', '3', '4', '5', '6', '7', '8', '9', '10']
education_clean_df['Education Bin'] = pd.cut(education_clean_df['MYS'], bins=education_bins, labels=education_bin_labels)
education_clean_df.head()

C:\Users\corey\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
C:\Users\corey\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """


,County ID,County Name,Perc < High School,Perc High School,Perc Associate,Perc Bachelors <,MYS,Education Bin
0,0,United States,13.0,27.5,29.1,30.3,14.734,9
1,1001,Autauga County,12.4,34.3,28.7,24.6,14.294,8
2,1003,Baldwin County,10.0,28.7,31.8,29.5,14.796,9
3,1005,Barbour County,26.2,34.9,26.0,12.9,13.028,1
4,1007,Bibb County,19.3,41.8,26.9,12.0,13.112,2


In [113]:
# Removing county 15005 from our education dataset as well, since it isn't included in our socio_econo file
education_clean_df = education_clean_df[education_clean_df['County ID'] != 15005]
education_clean_df.reset_index().head()

,index,County ID,County Name,Perc < High School,Perc High School,Perc Associate,Perc Bachelors <,MYS,Education Bin
0,0,0,United States,13.0,27.5,29.1,30.3,14.734,9
1,1,1001,Autauga County,12.4,34.3,28.7,24.6,14.294,8
2,2,1003,Baldwin County,10.0,28.7,31.8,29.5,14.796,9
3,3,1005,Barbour County,26.2,34.9,26.0,12.9,13.028,1
4,4,1007,Bibb County,19.3,41.8,26.9,12.0,13.112,2


In [114]:
education_clean_df['Education Bin'].value_counts()

1     328
4     316
10    315
3     315
8     314
6     314
5     314
9     313
7     312
2     301
Name: Education Bin, dtype: int64

In [115]:
socio_columns = ['State', 'County ID', 'County Name', '1', '2', '3', '4',' 5', '6','7', 'Median Household Income', '8', '9']
socio_econo_file.columns = socio_columns
socio_econo_file.head()

,State,County ID,County Name,1,2,3,4,5,6,7,Median Household Income,8,9
0,0,0,United States,310764925,47240290,46990033,47490547,15.2,15.1,15.3,54134,54052,54216
1,1,1001,Autauga County (AL),54905,7206,5914,8497,13.1,10.8,15.5,53748,49752,57745
2,1,1003,Baldwin County (AL),197091,25850,21888,29811,13.1,11.1,15.1,50976,48035,53917
3,1,1005,Barbour County (AL),23311,6731,5511,7952,28.9,23.6,34.1,32111,29316,34907
4,1,1007,Bibb County (AL),20316,4268,3359,5177,21.0,16.5,25.5,40113,36439,43787


In [116]:
# Change county name to just have county name, not state name, too.
socio_cleaned_columns = ['County ID', 'County Name', 'Median Household Income']
socio_econo_clean_df = socio_econo_file[socio_cleaned_columns]
socio_econo_clean_df.head()

,County ID,County Name,Median Household Income
0,0,United States,54134
1,1001,Autauga County (AL),53748
2,1003,Baldwin County (AL),50976
3,1005,Barbour County (AL),32111
4,1007,Bibb County (AL),40113


In [118]:
# Dropped the fucked row and reset index
socio_econo_clean_df = socio_econo_clean_df[socio_econo_clean_df['County ID'] != 15005]
socio_econo_clean_df.reset_index().head()

,index,County ID,County Name,Median Household Income
0,0,0,United States,54134
1,1,1001,Autauga County (AL),53748
2,2,1003,Baldwin County (AL),50976
3,3,1005,Barbour County (AL),32111
4,4,1007,Bibb County (AL),40113


In [119]:
print(socio_econo_clean_df.iloc[549:555, :])

     County ID           County Name Median Household Income
549      15001    Hawaii County (HI)                   51582
550      15003  Honolulu County (HI)                   74794
552      15007     Kauai County (HI)                   60725
553      15009      Maui County (HI)                   64971
554      16001       Ada County (ID)                   57133
555      16003     Adams County (ID)                   38973


In [120]:
# Now, we can convert Income column to integer
socio_econo_clean_df['Median Household Income'] = pd.to_numeric(socio_econo_clean_df['Median Household Income'], errors='coerce')

In [122]:
# We will create the same bins based on these percentile ranges and see if they match up with our education bins
socio_econo_clean_df['Income Bin'] = ""

income_bins = socio_econo_clean_df['Median Household Income'].describe(percentiles=[0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 
                                                                                    0.8, 0.9])[['min', '10%', '20%', '30%', 
                                                                                                '40%', '50%', '60%', '70%', 
                                                                                                '80%', '90%', 'max']]
income_bin_labels = ['1', '2', '3', '4', '5', '6', '7', '8', '9', '10']
socio_econo_clean_df['Income Bin'] = pd.cut(socio_econo_clean_df['Median Household Income'], 
                                            bins=income_bins, labels=income_bin_labels)
socio_econo_clean_df.head()

,County ID,County Name,Median Household Income,Income Bin
0,0,United States,54134,8
1,1001,Autauga County (AL),53748,8
2,1003,Baldwin County (AL),50976,8
3,1005,Barbour County (AL),32111,1
4,1007,Bibb County (AL),40113,3


In [124]:
socio_econo_clean_df = socio_econo_clean_df.drop_duplicates(['County ID'], keep='first')

In [126]:
print(education_clean_df.shape)
print(socio_econo_clean_df.shape)
# This shows that there are a few extra rows in our education dataset that will need to be addressed, as they will be occupied 
# by empty or NaN values

(3152, 8)
(3145, 4)


In [127]:
# Now, we should be able to merge everything...
education_income_df = pd.merge(education_clean_df, socio_econo_clean_df, on='County ID', how='outer', 
                               suffixes=('_ED', 'SE'), validate='1:1')

education_income_df.head()

,County ID,County Name_ED,Perc < High School,Perc High School,Perc Associate,Perc Bachelors <,MYS,Education Bin,County NameSE,Median Household Income,Income Bin
0,0,United States,13.0,27.5,29.1,30.3,14.734,9,United States,54134.0,8
1,1001,Autauga County,12.4,34.3,28.7,24.6,14.294,8,Autauga County (AL),53748.0,8
2,1003,Baldwin County,10.0,28.7,31.8,29.5,14.796,9,Baldwin County (AL),50976.0,8
3,1005,Barbour County,26.2,34.9,26.0,12.9,13.028,1,Barbour County (AL),32111.0,1
4,1007,Bibb County,19.3,41.8,26.9,12.0,13.112,2,Bibb County (AL),40113.0,3


In [128]:
education_income_clean_df = education_income_df[['County ID', 'MYS', 'Education Bin',
       'County NameSE', 'Median Household Income', 'Income Bin']]
education_income_clean_df.head()

,County ID,MYS,Education Bin,County NameSE,Median Household Income,Income Bin
0,0,14.734,9,United States,54134.0,8
1,1001,14.294,8,Autauga County (AL),53748.0,8
2,1003,14.796,9,Baldwin County (AL),50976.0,8
3,1005,13.028,1,Barbour County (AL),32111.0,1
4,1007,13.112,2,Bibb County (AL),40113.0,3


In [129]:
# Move county name to front and rename
education_income_clean_df = education_income_clean_df[['County NameSE', 'County ID',  'MYS', 'Education Bin',
       'Median Household Income', 'Income Bin']]
education_income_clean_df = education_income_clean_df.rename(columns={'County NameSE' : 'County Name (State)'})
education_income_clean_df.head()

,County Name (State),County ID,MYS,Education Bin,Median Household Income,Income Bin
0,United States,0,14.734,9,54134.0,8
1,Autauga County (AL),1001,14.294,8,53748.0,8
2,Baldwin County (AL),1003,14.796,9,50976.0,8
3,Barbour County (AL),1005,13.028,1,32111.0,1
4,Bibb County (AL),1007,13.112,2,40113.0,3


In [27]:
education_income_clean_df.shape
# This means there are more rows than either of the two original dataframes had initially...  drop those NaN's.

(3154, 10)

In [28]:
education_income_clean_df = education_income_clean_df.dropna()
education_income_clean_df.shape

(3141, 10)

In [130]:
education_income_clean_df['Income Bin'] = pd.to_numeric(education_income_clean_df['Income Bin']).astype(float)
education_income_clean_df['Education Bin'] = pd.to_numeric(education_income_clean_df['Education Bin']).astype(float)
education_income_clean_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3154 entries, 0 to 3153
Data columns (total 6 columns):
County Name (State)        3145 non-null object
County ID                  3154 non-null int64
MYS                        3142 non-null float64
Education Bin              3142 non-null float64
Median Household Income    3145 non-null float64
Income Bin                 3144 non-null float64
dtypes: float64(4), int64(1), object(1)
memory usage: 172.5+ KB


In [131]:
education_income_clean_df['County ID'].count()

3154

In [ ]:
# Chi square test comparing edu against income
#critical_value = stats.chi2.ppf(q=0.99, df=3140)
#critical_value

In [132]:
#stats.chisquare(education_income_clean_df['Education Bin'], education_income_clean_df['Income Bin'])

In [133]:
stats.ttest_ind(education_income_clean_df['Education Bin'], education_income_clean_df['Income Bin'])

C:\Users\corey\Anaconda3\lib\site-packages\scipy\stats\_distn_infrastructure.py:879: RuntimeWarning: invalid value encountered in greater
  return (self.a < x) & (x < self.b)
C:\Users\corey\Anaconda3\lib\site-packages\scipy\stats\_distn_infrastructure.py:879: RuntimeWarning: invalid value encountered in less
  return (self.a < x) & (x < self.b)
C:\Users\corey\Anaconda3\lib\site-packages\scipy\stats\_distn_infrastructure.py:1821: RuntimeWarning: invalid value encountered in less_equal
  cond2 = cond0 & (x <= self.a)


Ttest_indResult(statistic=nan, pvalue=nan)

In [134]:
mortality_clean = mortality_file[['County', 'County Code', 'Crude Rate']]
mortality_clean = mortality_clean.rename(columns={'County Code':'County ID'})
mortality_clean.head()
mortality_clean.shape

(2896, 3)

In [135]:
mortality_clean['Mortality Bin'] = ""
mortality_clean['Crude Rate'] = pd.to_numeric(mortality_clean['Crude Rate'], errors='Unreliable')
mortality_bins = mortality_clean['Crude Rate'].describe(percentiles=[0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 
                                                                                    0.8, 0.9])[['min', '10%', '20%', '30%', 
                                                                                                '40%', '50%', '60%', '70%', 
                                                                                                '80%', '90%', 'max']]
mortality_bin_labels = ['1', '2', '3', '4', '5', '6', '7', '8', '9', '10']
mortality_clean['Mortality Bin'] = pd.cut(mortality_clean['Crude Rate'], 
                                            bins=mortality_bins, labels=mortality_bin_labels)
mortality_clean.head()

,County,County ID,Crude Rate,Mortality Bin
0,"Autauga County, AL",1001,130.0,6
1,"Baldwin County, AL",1003,113.0,5
2,"Barbour County, AL",1005,165.1,8
3,"Bibb County, AL",1007,195.8,9
4,"Blount County, AL",1009,130.7,7


In [136]:
mortality_clean.shape

(2896, 4)

In [137]:
combined_data = pd.merge(education_income_clean_df, mortality_clean, how='outer', on='County ID', suffixes=('_EI', 'MORT'))

In [138]:
combined_data.dropna()
del combined_data['County Name (State)']
combined_data.shape

(3154, 8)

In [139]:
combined_data.head()

,County ID,MYS,Education Bin,Median Household Income,Income Bin,County,Crude Rate,Mortality Bin
0,0,14.734,9.0,54134.0,8.0,NaN,NaN,NaN
1,1001,14.294,8.0,53748.0,8.0,"Autauga County, AL",130.0,6
2,1003,14.796,9.0,50976.0,8.0,"Baldwin County, AL",113.0,5
3,1005,13.028,1.0,32111.0,1.0,"Barbour County, AL",165.1,8
4,1007,13.112,2.0,40113.0,3.0,"Bibb County, AL",195.8,9


In [140]:
combined_data = combined_data[['County', 'County ID', 'MYS', 'Education Bin', 'Median Household Income', 
                               'Income Bin', 'Crude Rate', 'Mortality Bin']]
combined_data = combined_data.dropna()
combined_data.shape
combined_data.head()

,County,County ID,MYS,Education Bin,Median Household Income,Income Bin,Crude Rate,Mortality Bin
1,"Autauga County, AL",1001,14.294,8.0,53748.0,8.0,130.0,6
2,"Baldwin County, AL",1003,14.796,9.0,50976.0,8.0,113.0,5
3,"Barbour County, AL",1005,13.028,1.0,32111.0,1.0,165.1,8
4,"Bibb County, AL",1007,13.112,2.0,40113.0,3.0,195.8,9
5,"Blount County, AL",1009,13.328,3.0,46377.0,6.0,130.7,7


In [141]:
combined_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2652 entries, 1 to 3151
Data columns (total 8 columns):
County                     2652 non-null object
County ID                  2652 non-null int64
MYS                        2652 non-null float64
Education Bin              2652 non-null float64
Median Household Income    2652 non-null float64
Income Bin                 2652 non-null float64
Crude Rate                 2652 non-null float64
Mortality Bin              2652 non-null category
dtypes: category(1), float64(5), int64(1), object(1)
memory usage: 168.7+ KB


In [142]:
combined_data['Mortality Bin'] = pd.to_numeric(combined_data['Mortality Bin']).astype(float)
combined_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2652 entries, 1 to 3151
Data columns (total 8 columns):
County                     2652 non-null object
County ID                  2652 non-null int64
MYS                        2652 non-null float64
Education Bin              2652 non-null float64
Median Household Income    2652 non-null float64
Income Bin                 2652 non-null float64
Crude Rate                 2652 non-null float64
Mortality Bin              2652 non-null float64
dtypes: float64(6), int64(1), object(1)
memory usage: 186.5+ KB


In [143]:
combined_data.shape

(2652, 8)

In [147]:
combined_data.to_csv('CJK-Combined_Dataset.csv', index=False)

In [144]:
from scipy.stats import linregress

In [145]:
stats.linregress(combined_data['Education Bin'], combined_data['Income Bin'])

LinregressResult(slope=0.7059097662053324, intercept=1.6173438110876552, rvalue=0.7160071850186737, pvalue=0.0, stderr=0.01336971348072304)